# Day 01: Data Access Control in Python Classes

**What I'm learning today:**
- How to control access to class attributes
- Properties vs getters/setters
- Static attributes and methods
- Python's approach to privacy (it's different from Java!)

**Why this matters:** By default, anyone can read/write my class data. I need to learn how to protect and validate it properly.




Python’s Take on Access Modifiers
Unlike languages such as Java or C++, which enforce strict access control (like private or protected), Python takes a more relaxed approach. In Python:

A single underscore (_) before a name (e.g., _attribute) is a convention indicating that something is intended for internal use within the class or module. This means it’s not part of the public API, and external code shouldn’t access it directly.

However, Python doesn’t enforce this restriction. The attribute or method is still accessible from outside the class, but it signals to developers that it’s meant to be “protected” or “internal.”

The “Consenting Adults” Philosophy
Guido van Rossum’s "consenting adults" philosophy highlights Python’s emphasis on developer responsibility rather than strict rules. This philosophy suggests that:

Developers are trusted to respect the convention of not accessing underscore-prefixed attributes or methods.
Access is not prevented, as Python assumes that developers will act responsibly and won’t misuse or access “protected” members unless absolutely necessary.






















In [ ]:
# Example: No protection = chaos
# This is what happens when I don't control access - anyone can mess with my data
class User:
    def __init__(self, username, email, password):
        self.username = username
        self.email = email
        self.password = password

user1 = User("Amir", "Amir@gmail.com", "123")
print(f"Original email: {user1.email}")

# Oops! Someone can set an invalid email and I can't stop them
user1.email = "Amiroutlook.com"  # Missing @ symbol!
print(f"After messing with it: {user1.email}")

# Even worse - they can see the password!
print(f"Password exposed: {user1.password}")


## Solution 1: Getters & Setters (Java-Style)

**When I'd use this:** If I'm coming from Java/C++ or need explicit control.

**How it works:** Hide the real attribute (use `_email`) and provide `getEmail()` and `setEmail()` methods.

**Pros:** Explicit, clear what's happening  
**Cons:** Verbose, not very "Pythonic"


In [ ]:
from datetime import datetime

class UserWithGetters:
    def __init__(self, username, email, is_admin=False):
        self.username = username
        self._email = email  # _ prefix = "internal use only" (convention, not enforced!)
        self.is_admin = is_admin

    def get_email(self):
        """Getter: Control how email is accessed"""
        if self.is_admin:
            print(f"Email accessed at {datetime.now()}")
            return self._email
        return None  # Non-admins can't see email

    def set_email(self, new_email):
        """Setter: Validate before setting"""
        if "@" in new_email:
            self._email = new_email
        else:
            raise ValueError("Invalid email: must contain '@'")

# Testing this out
user = UserWithGetters("alice", "alice@example.com", is_admin=True)
print(f"Email via getter: {user.get_email()}")

# Try to set invalid email
try:
    user.set_email("invalid-email")
except ValueError as e:
    print(f"Error: {e}")

# I CAN still access _email directly (Python doesn't stop me)
# But I shouldn't! It's a convention, not a rule.
print(f"Direct access (naughty!): {user._email}")


### Quick Note: Python's "Consenting Adults" Philosophy

Python doesn't enforce privacy like Java/C++. Instead:

- `_attribute` = "protected" (convention: internal use, but accessible)
- `__attribute` = "private" (name mangling, but still accessible if I try)
- No underscore = public

**The rule:** Python trusts me to respect conventions. I shouldn't access `_` prefixed stuff from outside the class unless I really need to.


## Solution 2: Properties (The Python Way) ⭐

**When I'd use this:** Almost always. This is the recommended Python approach.

**How it works:** Use `@property` decorator. Looks like normal attribute access (`user.email`) but runs custom code.

**Pros:** Clean syntax, Pythonic, looks like normal attributes  
**Cons:** Slightly more magic (but worth it!)


In [ ]:
class UserWithProperties:
    def __init__(self, username, email, is_admin=False):
        self.username = username
        self.is_admin = is_admin
        self.email = email  # This triggers the setter!

    @property
    def email(self):
        """Getter: Called when I do user.email"""
        if self.is_admin:
            return self._email
        return None  # Non-admins get None

    @email.setter
    def email(self, new_email):
        """Setter: Called when I do user.email = 'something'"""
        if "@" not in new_email:
            raise ValueError(f"Invalid email '{new_email}': must contain '@'")
        self._email = new_email

# Testing - looks like normal attribute access!
user = UserWithProperties("bob", "bob@example.com", is_admin=True)
print(f"Email: {user.email}")  # Calls the getter

# Setting email - looks normal but validates!
user.email = "bob.new@example.com"
print(f"New email: {user.email}")

# Invalid email raises error
try:
    user.email = "not-an-email"
except ValueError as e:
    print(f"Error: {e}")


### Bonus: Read-Only Properties

Want an attribute that can be read but never changed? Just don't add a setter! Easy.


In [ ]:
class Product:
    def __init__(self, name, price):
        self.name = name
        self._price = price
        self._created_at = datetime.now()

    @property
    def price(self):
        """Read-only: price can't be changed after creation"""
        return self._price

    @property
    def created_at(self):
        """Read-only: creation time is immutable"""
        return self._created_at

    @property
    def display_name(self):
        """Computed property: combines name and price"""
        return f"{self.name} - ${self._price:.2f}"

product = Product("Laptop", 999.99)
print(f"Price: ${product.price}")
print(f"Created: {product.created_at}")
print(f"Display: {product.display_name}")

# Try to change price - won't work (no setter = read-only)
try:
    product.price = 799.99
except AttributeError as e:
    print(f"Error: {e}")


## Static Attributes & Methods

**What:** Attributes/methods that belong to the CLASS, not individual instances.

**When I'd use this:** Track class-level data (like "how many users created?") or utility functions.

**How:** Define them at class level (not in `__init__`). Access via `ClassName.attribute` or `ClassName.method()`.


In [ ]:
class User:
    # Static attribute: shared by ALL instances
    total_users = 0
    admin_count = 0
    
    def __init__(self, username, email, is_admin=False):
        self.username = username
        self.email = email
        self.is_admin = is_admin
        
        # Increment class-level counter
        User.total_users += 1
        if is_admin:
            User.admin_count += 1
    
    @staticmethod
    def get_stats():
        """Static method: doesn't need 'self', belongs to class"""
        return {
            "total": User.total_users,
            "admins": User.admin_count,
            "regular": User.total_users - User.admin_count
        }
    
    @classmethod
    def create_admin(cls, username, email):
        """Class method: alternative constructor"""
        return cls(username, email, is_admin=True)

# Before creating users
print(f"Initial stats: {User.get_stats()}")

# Create some users
user1 = User("alice", "alice@example.com")
user2 = User("bob", "bob@example.com", is_admin=True)
user3 = User.create_admin("charlie", "charlie@example.com")  # Using class method

# Check stats
print(f"After creating users: {User.get_stats()}")
print(f"Total users (via class): {User.total_users}")
print(f"Total users (via instance): {user1.total_users}")  # Same value!


### Static vs Class Methods - Quick Note

- `@staticmethod`: Doesn't need `self` or `cls`. Just a regular function that happens to live in the class.
- `@classmethod`: Gets `cls` as first param. Use for alternative constructors or when I need the class itself.

**Example:**
```python
@staticmethod
def is_valid_email(email):  # No self/cls needed
    return "@" in email

@classmethod
def from_dict(cls, data):  # Gets cls, can create instances
    return cls(data['username'], data['email'])
```


## Quick Reference: When to Use What?

| Situation | Use This |
|-----------|----------|
| Need validation when setting? | `@property` with setter |
| Need to control access (read-only)? | `@property` without setter |
| Need computed/derived values? | `@property` (no storage needed) |
| Coming from Java/C++? | Getters/setters (but I should learn properties!) |
| Track class-level data? | Static attributes |
| Utility functions related to class? | `@staticmethod` or `@classmethod` |

**Bottom line:** Use `@property` for 90% of cases. It's Pythonic and clean.


## Key Takeaways

1. **Python doesn't enforce privacy** - it's a convention (`_` prefix), not a rule
2. **Properties are the Python way** - use `@property` instead of getters/setters
3. **Static = class-level** - shared by all instances, not per-instance
4. **Read-only = no setter** - just define the getter property
5. **Trust but verify** - Python trusts me to respect conventions

**Remember:** I'm a consenting adult. Don't access `_` prefixed stuff unless I really need to!
